In [ ]:
'''monthly robberies data set'''
import pandas as pd
import math
from sklearn.metrics import mean_squared_error as mse
from math import sqrt
from matplotlib import pyplot as plt
#load data set
data_series = pd.read_csv('./data/robberies.csv', header=0, index_col=0, parse_dates=True, squeeze=True)

#split data set into model development data set and cross validation set, use last year of data as cv
data_end = len(data_series) - 12
data_set, cv_set = data_series[0:data_end], data_series[data_end::]
print(f'Data set: {len(data_set)} months, Validation set: {len(cv_set)} months')

#save training and cv sets to csv format
data_set.to_csv('data_set.csv', header = False)
cv_set.to_csv('cv_set.csv', header = False)

In [ ]:
#create training set and test set
X = data_set.values
X = X.astype('float32')

#start with train set of 50%
train_size = int(0.5*len(X))
train_set, test_set = X[0:train_size], X[train_size::]

In [ ]:
#use walk forward validation to create baseline prediction forecast using the persistence model
#create foreacst history
historic_obs = [x for x in train_set]
#create baseline predictions
predictions = []
for i in range(len(test_set)):
    #make prediction at t using observation at t-1
    y_hat = historic_obs[-1]
    predictions.append(y_hat)
    #actual t observation from test set
    observation = test_set[i]
    #update historic observations with actual t observation
    historic_obs.append(observation)
    print(f'Predicted: {y_hat: .3f}, Expected: {observation: .3f}')
#report the performance of the forecast using RMSE
rmse = sqrt(mse(test_set, predictions))
print(f'RMSE: {rmse: .3f}')

In [ ]:
# show summary statistics of data_set time series
print(data_set.describe())

In [ ]:
#plot the data
data_set.plot()
plt.show()

### Line plot observations
- There is a trend of increasing robberies
- There is high variability among data points, which corresponds to the wide spread between the quartile ranges observed above
- The variance between data points seem to increase over time, as observed from the widening ampitude of the fluctuations in the data
- The data set is non-stationary

In [ ]:
#plot histogram and denisty plot
plt.figure(1)
plt.subplot(211)
data_set.hist()
plt.subplot(212)
data_set.plot(kind='kde')
plt.show()

### Histogram and Density plot observations
- The data distribution is not gaussian
- The data seem to be quadratic or exponential, due to the left shift and the short right tail


In [ ]:
#group data by year for box plot analysis
#1974 only has 10 months and not a full year, so we exclude it
grps = data_set['1966':'1973'].groupby(pd.Grouper(freq='A'))
yrs = pd.DataFrame({name.year: grp.values for name, grp in grps})
yrs.boxplot()
plt.show()

### Box plot observations
- The earlier 2 years seem to have a much smaller variance, with the 1st and 3rd quantile being much closer to the median values.
- The variance changes over time, but does not appear to do so consistently.
- The median values do not exhibit a linear trend.

In [ ]:
#Implement an ARIMA model to forecast the number of robberies over time
#first check for stationarity of the data set using the augmented Dickey-Fuller test
from statsmodels.tsa.stattools import adfuller

def check_stationary(data):
    '''function to check data stationarity using adfuller from statsmodel'''
    result = adfuller(data)
    print(f'ADF Statistic: {result[0]: .3f}')
    print(f'p-value: {result[1]: .3f}')
    print(f'Critical Values:')
    for key, value in result[4].items():
        print('\t%s: %.3f' % (key, value))


#check if stationary using Dickey-Fuller
check_stationary(data_set.values)

### Dickey-Fuller Test observation:
- The test statistic value is 0.797 , which is larger than the critical value of -2.893. This indicates that we cannot reject the null hypothesis, which claims that the data is non-stationary, with a significance level of less than 5%.

In [ ]:
#Employ first order differencing
def differencing(data):
    '''Function executes differencing of order 1 when called'''
    differenced = [(data[i] - data[i-1]) for i in range(1, len(data))]
    return differenced

#executing differencing of order 1
stationary_X = differencing(X)

#check if the data is stationary now
check_stationary(stationary_X)

- The test statistic value is -3.981 , which is smaller than the critical value of -2.893. This indicates that we can reject the null hypothesis, which claims that the data is non-stationary, with a significance level of less than 5%.
- Differencing by order of 1 seems to have made the data set stationary.

In [ ]:
#create ACF and PACF plots to check for autocorrelation
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
plt.figure()
plt.subplot(211)
plot_acf(X, lags=50, ax=plt.gca())
plt.subplot(212)
plot_pacf(X, lags=50, ax=plt.gca())

#fix any plot overlap with tight layout
plt.tight_layout()
plt.show()

### ACF and PACF plot observations

- The ACF plot shows that lags are significant for 10-12 months.
- The PACF plot shows that lags are significant for likely just 2 months.
- The PACF plot suggests that perhaps the autocorrelations at lag 3 and thereafter are due to the propagation of the autocorrelations at lags 1 and 2.
- The plot suggest an ARMA(12,2) that can be used for modelling.
- Since differencing order of the data is 1, an ARIMA(12,1,2) should be a good starting point for modelling the data.

In [ ]:
#try an ARIMA model
from statsmodels.tsa.arima_model import ARIMA
arima1212_hist = [x for x in train_set]
arima1212_pred = []
for i in range(len(test_set)):
    # predict
    model = ARIMA(arima1212_hist, order=(0,1,2))
    model_fit = model.fit(disp=0)
    arima1212_y_hat = model_fit.forecast()[0]
    arima1212_pred.append(arima1212_y_hat)
    # observation
    obs = test_set[i]
    arima1212_hist.append(obs)
    print('>Predicted=%.3f, Expected=%.3f' % (arima1212_y_hat, obs))
# report performance
rmse = sqrt(mse(test_set, arima1212_pred))
print(f'RMSE: {rmse: .3f}')

In [ ]:
#Use Grid Search to find the optimal p,d,q hyperparameters
def evaluate_arima_model(data, arima_order):
    '''Function to evaluate data set X based on ARIMA order with some (p,d,q) and return RMSE'''
    #prepare training data
    data = data.astype('float32')
    train_size = int(len(data) * 0.50)
    train, test = data[0:train_size], data[train_size:]
    history = [x for x in train]
    # make predictions
    predictions = list()
    for t in range(len(test)):
        model = ARIMA(history, order=arima_order)
        model_fit = model.fit(disp=0)
        yhat = model_fit.forecast()[0]
        predictions.append(yhat)
        history.append(test[t])
    # calculate out of sample error
    rmse = sqrt(mse(test, predictions))
    return rmse

def evaluate_models(data, p_val, d_val, q_val):
    #convert data set to float to prevent numpy error
    data = data.astype('float32')
    best_rmse, best_config = float('inf'), None
    #loop through all values of p,d,q to try every configuration order for ARIMA
    for p in p_val:
        for d in d_val:
            for q in q_val:
                order = (p,d,q)
                try:
                    rmse = evaluate_arima_model(data, order)
                    if rmse < best_rmse:
                        best_rmse, best_config = rmse, order
                    print('ARIMA%s, RMSE = %.3f' % (order, rmse))
                except:
                    continue
    print('Best ARIMA%s, RMSE = %.3f' % (best_config, best_rmse))

In [ ]:
#evaluate data set with all possible hyperparameter configurations
p_val = range(0,13)
d_val = range(0,2)
q_val = range(0,13)

#turn off verbose warning messages
import warnings
warnings.filterwarnings("ignore")
evaluate_models(X, p_val, d_val, q_val)